<div class="alert alert-block alert-success">
<em>Panelite</em> is powered by young technologies like <a href="https://pyodide.org/en/stable/">Pyodide</a> and <a href="https://jupyterlite.readthedocs.io/en/latest/">Jupyterlite</a>. Some browsers may be poorly supported (e.g. mobile or 32-bit versions). If you experience issues, please <a href="https://github.com/holoviz/panel/issues">report them</a>.
</div>

In [ ]:
import piplite
await piplite.install(['altair'])

In [ ]:
import altair as alt
import pandas as pd
import panel as pn

pn.extension('vega', template='fast')

pn.state.template.title = "Altair Brushing Example"

This example demonstrates how to leverage the brushing/linked-selections support in the Vega pane to update a table.

In [ ]:
df = pd.read_json("https://raw.githubusercontent.com/vega/vega/master/docs/data/penguins.json")

brush = alt.selection_interval(name='brush')  # selection of type "interval"

chart = alt.Chart(df).mark_point().encode(
    x=alt.X('Beak Length (mm):Q', scale=alt.Scale(zero=False)),
    y=alt.Y('Beak Depth (mm):Q', scale=alt.Scale(zero=False)),
    color=alt.condition(brush, 'Species:N', alt.value('lightgray'))
).properties(
    width=300,
    height=300
).add_params(
    brush
)

vega_pane = pn.pane.Vega(chart, debounce=10)

def filtered_table(selection):
    if not selection:
        return df.iloc[:0]
    query = ' & '.join(
        f'{crange[0]:.3f} <= `{col}` <= {crange[1]:.3f}'
        for col, crange in selection.items()
    )
    return df.query(query)

pn.Column(
    'Select points on the plot and watch the linked table update.',
    sizing_mode='stretch_width'
).servable()

pn.Row(
    vega_pane,
    pn.Column(
        pn.pane.DataFrame(
            pn.bind(filtered_table, vega_pane.selection.param.brush)
        ),
        height=350
    )
).servable()